In [1]:
 # Import libraries
import pandas as pd
import pickle as pk

# import libraries for api
import os
from dotenv import load_dotenv 
import requests
import json

# import libraries for vizualization
import plotly.express as px
import panel as pn
from pprint import pprint

In [2]:
### Open data from a pickle : Zillow_data, which represent the houses currently for sale
filename = "Zillow_data"
infile = open(filename,'rb')
new_dict = pk.load(infile)
infile.close()

In [4]:
### transform to json
z_for_sale_resp =new_dict.json()

In [5]:
### transform json to datafram
current_zillow_df = pd.json_normalize(data=z_for_sale_resp["props"])
print('num of rows:', len(current_zillow_df))
print('num of columns:', len(current_zillow_df.columns))

current_zillow_df.head()

num of rows: 40
num of columns: 18


,bathrooms,propertyType,lotAreaValue,address,imgSrc,price,listingDateTime,listingStatus,zpid,longitude,latitude,lotAreaUnit,daysOnZillow,bedrooms,country,currency,livingArea,hasImage
0,2,SINGLE_FAMILY,1742.40,"318 W Ruscomb St, Philadelphia, PA 19120",https://photos.zillowstatic.com/fp/c528e4507f6...,150000,None,FOR_SALE,10428068,-75.130410,40.026787,sqft,-1,4,USA,USD,1440,True
1,2,SINGLE_FAMILY,3920.40,"4913 Castor Ave, Philadelphia, PA 19124",https://photos.zillowstatic.com/fp/84229141ec1...,325000,None,FOR_SALE,10291269,-75.091110,40.024803,sqft,-1,4,USA,USD,1920,True
2,4,SINGLE_FAMILY,0.43,"200 W Walnut Ln, Philadelphia, PA 19144",https://photos.zillowstatic.com/fp/b90b6b03745...,869000,None,FOR_SALE,2068504637,-75.181816,40.037050,acres,-1,5,USA,USD,3857,True
3,4,SINGLE_FAMILY,9583.20,"2320 N 50th St, Philadelphia, PA 19131",https://photos.zillowstatic.com/fp/458505836a3...,267000,None,FOR_SALE,10486635,-75.227104,39.996180,sqft,-1,5,USA,USD,4520,True
4,3,SINGLE_FAMILY,3049.20,"522 Gilham St, Philadelphia, PA 19111",https://photos.zillowstatic.com/fp/04cc332dd77...,144000,None,FOR_SALE,2071261604,-75.092926,40.050960,sqft,-1,5,USA,USD,2007,True


In [10]:
### make a list of the zpid
zpid_list = current_zillow_df["zpid"].tolist()

In [11]:
### Select columns that we will need for the analysis
current_zillow_df.columns
current_price_df = current_zillow_df[[ "zpid","propertyType" , "price", "livingArea" , "bathrooms" , "bedrooms" ,"address", "listingStatus" , "latitude" , "longitude", "imgSrc"]].copy()
current_price_df.set_index("zpid", inplace=True)


#Pull only the zip code from the address column
current_price_df["address"] = current_price_df["address"].str[-5:]

# Change the column names to improve readability
current_price_df.rename(columns = {
        "livingArea" : "Area in sqft.",
        "propertyType" : "Type of Property",
        "address" : "zip code",
        "listingStatus" : "Status",
        "imgSrc" : "Image property"
        }, inplace = True)

current_price_df.head(17)

,Type of Property,price,Area in sqft.,bathrooms,bedrooms,zip code,Status,latitude,longitude,Image property
zpid,,,,,,,,,,
10428068,SINGLE_FAMILY,150000,1440,2,4,19120,FOR_SALE,40.026787,-75.130410,https://photos.zillowstatic.com/fp/c528e4507f6...
10291269,SINGLE_FAMILY,325000,1920,2,4,19124,FOR_SALE,40.024803,-75.091110,https://photos.zillowstatic.com/fp/84229141ec1...
2068504637,SINGLE_FAMILY,869000,3857,4,5,19144,FOR_SALE,40.037050,-75.181816,https://photos.zillowstatic.com/fp/b90b6b03745...
10486635,SINGLE_FAMILY,267000,4520,4,5,19131,FOR_SALE,39.996180,-75.227104,https://photos.zillowstatic.com/fp/458505836a3...
2071261604,SINGLE_FAMILY,144000,2007,3,5,19111,FOR_SALE,40.050960,-75.092926,https://photos.zillowstatic.com/fp/04cc332dd77...
10581168,SINGLE_FAMILY,309999,1254,2,3,19154,FOR_SALE,40.084550,-74.979710,https://photos.zillowstatic.com/fp/41a70b40f7e...
10344057,SINGLE_FAMILY,260000,1350,2,3,19151,FOR_SALE,39.972546,-75.238495,https://photos.zillowstatic.com/fp/048835ff285...
10571315,SINGLE_FAMILY,210000,1350,3,5,19136,FOR_SALE,40.042736,-75.026120,https://photos.zillowstatic.com/fp/3e8f02c9c85...
10522272,SINGLE_FAMILY,315000,1392,2,3,19152,FOR_SALE,40.065180,-75.031845,https://photos.zillowstatic.com/fp/48b79ad5717...


In [ ]:
### Open data from a pickle : Zillow_old_data, which represent the houses recently sold and similar to the current one for sale
filename = "Zillow_old_data"
infile = open(filename,'rb')
new_source = pk.load(infile)
infile.close()

In [13]:
### transform to json
z_recently_sold_resp =new_source.json()

In [14]:
### transform json to datafram
sold_zillow_df = pd.json_normalize(data=z_recently_sold_resp)
print('num of rows:', len(sold_zillow_df))
print('num of columns:', len(sold_zillow_df.columns))

sold_zillow_df.head()

num of rows: 5
num of columns: 18


,dateSold,bedrooms,homeStatus,latitude,miniCardPhotos,zpid,longitude,bathrooms,livingArea,livingAreaUnits,homeType,currency,lastSoldPrice,price,address.city,address.state,address.streetAddress,address.zipcode
0,1610582400000,5,RECENTLY_SOLD,40.055460,[{'url': 'https://photos.zillowstatic.com/fp/1...,10214276,-75.195090,3,2138,Square Feet,SINGLE_FAMILY,USD,470000,470000,Philadelphia,PA,7139 Lincoln Dr,19119
1,1619740800000,3,RECENTLY_SOLD,40.051693,[{'url': 'https://photos.zillowstatic.com/fp/f...,10284714,-75.194620,4,2200,Square Feet,SINGLE_FAMILY,USD,477500,477500,Philadelphia,PA,403 W Ellet St,19119
2,1614556800000,4,RECENTLY_SOLD,40.051390,[{'url': 'https://photos.zillowstatic.com/fp/6...,10284790,-75.195480,3,2400,Square Feet,SINGLE_FAMILY,USD,443000,443000,Philadelphia,PA,426 Glen Echo Rd,19119
3,1629158400000,5,RECENTLY_SOLD,40.057297,[{'url': 'https://photos.zillowstatic.com/fp/c...,103779525,-75.193504,3,2665,Square Feet,SINGLE_FAMILY,USD,480000,480000,Philadelphia,PA,7141 Cresheim Rd,19119
4,1634256000000,3,RECENTLY_SOLD,40.051323,[{'url': 'https://photos.zillowstatic.com/fp/4...,333709815,-75.196860,3,1760,Square Feet,SINGLE_FAMILY,USD,460000,460000,Philadelphia,PA,504 W Mount Pleasant Ave,19119


In [15]:
### Select columns that we will need for the analysis
sold_zillow_df.columns
sold_price_df = sold_zillow_df[[ "zpid", "homeType", "lastSoldPrice", "livingArea" , "bathrooms" , "bedrooms" , "address.zipcode",  "homeStatus" , "latitude" , "longitude", "miniCardPhotos"]].copy()
sold_price_df.set_index("zpid", inplace=True)

# Change the column names to improve readability
sold_price_df.rename(columns = {
        "homeType" : "Type of Property",
        "lastSoldPrice" : "Price",
        "livingArea" : "Area in sqft.",
        "address.zipcode" :"zip code",
        "homeStatus" : "Status",
        "miniCardPhotos" : "Image property"
        }, inplace = True)

sold_price_df.head(40)

,Type of Property,Price,Area in sqft.,bathrooms,bedrooms,zip code,Status,latitude,longitude,Image property
zpid,,,,,,,,,,
10214276,SINGLE_FAMILY,470000,2138,3,5,19119,RECENTLY_SOLD,40.055460,-75.195090,[{'url': 'https://photos.zillowstatic.com/fp/1...
10284714,SINGLE_FAMILY,477500,2200,4,3,19119,RECENTLY_SOLD,40.051693,-75.194620,[{'url': 'https://photos.zillowstatic.com/fp/f...
10284790,SINGLE_FAMILY,443000,2400,3,4,19119,RECENTLY_SOLD,40.051390,-75.195480,[{'url': 'https://photos.zillowstatic.com/fp/6...
103779525,SINGLE_FAMILY,480000,2665,3,5,19119,RECENTLY_SOLD,40.057297,-75.193504,[{'url': 'https://photos.zillowstatic.com/fp/c...
333709815,SINGLE_FAMILY,460000,1760,3,3,19119,RECENTLY_SOLD,40.051323,-75.196860,[{'url': 'https://photos.zillowstatic.com/fp/4...


In [51]:
total_price_df = pd.concat([current_price_df,sold_price_df ], axis=0, join="inner")
total_price_df.head()


,Type of Property,Area in sqft.,bathrooms,bedrooms,zip code,Status,latitude,longitude,Image property
zpid,,,,,,,,,
10428068,SINGLE_FAMILY,1440,2,4,19120,FOR_SALE,40.026787,-75.130410,https://photos.zillowstatic.com/fp/c528e4507f6...
10291269,SINGLE_FAMILY,1920,2,4,19124,FOR_SALE,40.024803,-75.091110,https://photos.zillowstatic.com/fp/84229141ec1...
2068504637,SINGLE_FAMILY,3857,4,5,19144,FOR_SALE,40.037050,-75.181816,https://photos.zillowstatic.com/fp/b90b6b03745...
10486635,SINGLE_FAMILY,4520,4,5,19131,FOR_SALE,39.996180,-75.227104,https://photos.zillowstatic.com/fp/458505836a3...
2071261604,SINGLE_FAMILY,2007,3,5,19111,FOR_SALE,40.050960,-75.092926,https://photos.zillowstatic.com/fp/04cc332dd77...


In [88]:
# Calculate average price per square foot for properties grouped by zipcode
market_price_df = sold_price_df[["Price", "Area in sqft.", "zip code"]].copy()
market_price_df['Per sqft price']  =  market_price_df["Price"] / market_price_df["Area in sqft."]
avg_zipcode_price = market_price_df.groupby("zip code").mean().drop(columns =["Price", "Area in sqft."])
                                                                  
avg_zipcode_price.head()                                                        

,Per sqft price
zip code,
19119,212.587323


In [46]:
# Calculate average price per square foot for properties grouped by zipcode
market_bed_df = sold_price_df[["Price", "bedrooms", "zip code"]].copy()
market_bed_df.groupby(["zip code", "bedrooms"]).mean()

Price
zip code bedrooms        
19119    3         468750
         4         443000
         5         475000

In [97]:
zpids = "19119"
avg_zipcode_price.loc[zpids]
avg_zipcode_price

AttributeError: 'Series' object has no attribute 'get_value'

In [ ]:
def mean_per_zipcode(zipcode)
    avg_price_zip = avg_zipcode_price.loc["zip code"]
    return avg_price_zip


def calculator(zpid, current_price_df)
    # column of current price per sqft for houses to sale currently
    row = current_price_df.index("zpid")
    current_price_df['Per sqft price']  =  current_price_df["Price"] / current_price_df["Area in sqft."]
    
    for 
    if 
    

In [92]:
def mean_per_zipcode(zipcode):
    avg_price_neighborhood = avg_zipcode_price.loc[zipcode]
    return avg_price_neighborhood

mean_per_zipcode("19119")

Per sqft price    212.587323
Name: 19119, dtype: float64

In [65]:
zpids = "10285435"

row = current_price_df.loc[zpids].tolist()
avg = row[1] / row[2]
avg

if avg_current < avg_zipcode

231.30040322580646